In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import sweetviz as sv
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
import csv
def load_csv(filename):
    """Helper to load the tournament data from csv."""
    rows = []
    with open(filename) as f:
        reader = csv.DictReader(f)
        rows.extend(iter(reader))
    return rows

In [4]:
# load data from championship games
fp_2017 = '/Users/christopherfiore/Documents/GitHub/cwl-data/data/data-2017-08-13-champs.csv'
fp_2018 = '/Users/christopherfiore/Documents/GitHub/cwl-data/data/data-2018-08-19-champs.csv'
fp_2019 = '/Users/christopherfiore/Documents/GitHub/cwl-data/data/data-2019-08-18-champs.csv'

champs_2017 = pd.DataFrame(load_csv(fp_2017))
champs_2018 = pd.DataFrame(load_csv(fp_2018))
champs_2019 = pd.DataFrame(load_csv(fp_2019))

#add year label
champs_2017['year'] = 2017
champs_2018['year'] = 2018
champs_2019['year'] = 2019


In [5]:
# get list of metrics that appear year-over-year
common_metrics = list(set(champs_2017.columns) & set(champs_2018.columns) & set(champs_2019.columns))

#concat yearly dfs
champs_df = pd.concat([champs_2017[common_metrics], champs_2018[common_metrics], champs_2019[common_metrics]])

In [6]:
# yearly match count by player 
champs_df.groupby(['year', 'player'])['match id'].nunique().to_frame('match_count').sort_values(by=['year', 'match_count'], ascending=[True, False]).reset_index()

,year,player,match_count
0,2017,Aqua,35
1,2017,Crimsix,35
2,2017,Faccento,35
3,2017,Felony,35
4,2017,Formal,35
...,...,...,...
413,2019,Reaper,9
414,2019,Seany,9
415,2019,Willett,9
416,2019,BBCONor,3


In [7]:
#score distributions by player
hardpoint_df = champs_df.loc[champs_df['mode'] == 'Hardpoint']
hardpoint_df.head()

,win?,3-piece,duration (s),team,fave scorestreaks,+/-,bomb plants,mode,hill time (s),accuracy (%),hill defends,snd rounds,k/d,map,snd firstbloods,5-streak,4-piece,suicides,shots,bomb sneak defuses,deaths per 10min,4-streak,headshots,6-streak,deaths,series id,bomb pickups,7-streak,kills per 10min,year,hill captures,kills,team kills,bomb defuses,2-piece,hits,match id,player,assists,fave weapon,8+-streak,end time,score
0,L,0,480.0,Echo Fox,"Scarab, Trinity Rocket, Bombardment",-7,0,Hardpoint,8,17.5%,0,0,0.63,Retaliation,0,0,0,0,422,0,23.8,0,1,0,19,champs-pool-A-3,0,0,15.0,2017,2,12,0,0,2,74,3A6D1AC2-1118-11D3-A6EB-0CFE456F15BE,Methodz,5,NV4,0,2017-08-09 14:11:50 UTC,106
1,L,0,480.0,Echo Fox,"Scarab, Trinity Rocket, Bombardment",-1,0,Hardpoint,34,21.2%,0,0,0.95,Retaliation,0,0,0,0,458,0,25.0,1,0,0,20,champs-pool-A-3,0,0,23.8,2017,2,19,0,0,2,97,3A6D1AC2-1118-11D3-A6EB-0CFE456F15BE,mosh,5,NV4,0,2017-08-09 14:11:50 UTC,106
2,L,0,480.0,Echo Fox,"Scarab, Trinity Rocket, Bombardment",-11,0,Hardpoint,43,14.0%,3,0,0.52,Retaliation,0,0,0,0,508,0,28.8,0,2,0,23,champs-pool-A-3,0,0,15.0,2017,6,12,0,0,0,71,3A6D1AC2-1118-11D3-A6EB-0CFE456F15BE,MRuiz,6,KBAR-32,0,2017-08-09 14:11:50 UTC,106
3,L,0,480.0,Echo Fox,"Scarab, Trinity Rocket, Bombardment",-2,0,Hardpoint,35,18.7%,4,0,0.88,Retaliation,0,0,0,0,406,0,21.2,0,2,0,17,champs-pool-A-3,0,0,18.8,2017,6,15,0,0,1,76,3A6D1AC2-1118-11D3-A6EB-0CFE456F15BE,Replays,6,KBAR-32,0,2017-08-09 14:11:50 UTC,106
4,W,0,480.0,Epsilon,"Scarab, Trinity Rocket, Bombardment",+3,0,Hardpoint,33,17.0%,2,0,1.20,Retaliation,0,0,0,1,476,0,18.8,1,0,0,15,champs-pool-A-3,0,0,22.5,2017,1,18,1,0,1,81,3A6D1AC2-1118-11D3-A6EB-0CFE456F15BE,Dqvee,9,NV4,0,2017-08-09 14:11:50 UTC,250


In [8]:
#convert to float
hardpoint_df['hill time (s)'] = hardpoint_df['hill time (s)'].astype(float)
hardpoint_df['hill defends'] = hardpoint_df['hill defends'].astype(float)
hardpoint_df['k/d'] = hardpoint_df['k/d'].astype(float)

In [9]:
hardpoint_df.corr()

,hill time (s),hill defends,k/d,year
hill time (s),1.00,0.17,0.39,-0.03
hill defends,0.17,1.00,0.14,0.45
k/d,0.39,0.14,1.00,0.01
year,-0.03,0.45,0.01,1.00


In [10]:
px.scatter(
    hardpoint_df,
    x = 'hill time (s)',
    y = 'hill defends',
    facet_col= 'year',
    color='player',
    height = 900, 
    title = "Hardpoint: Hill Defends vs Hill time (s)"
)

In [11]:
px.scatter(
    hardpoint_df,
    x = 'hill time (s)',
    y = 'k/d',
    facet_col= 'year',
    color='player',
    height = 900, 
    title = "Hardpoint: k/d vs Hill time (s)"
)

In [15]:
snd_df = champs_df.loc[champs_df['mode'] == 'Search & Destroy']
snd_df.head()

,win?,3-piece,duration (s),team,fave scorestreaks,+/-,bomb plants,mode,hill time (s),accuracy (%),hill defends,snd rounds,k/d,map,snd firstbloods,5-streak,4-piece,suicides,shots,bomb sneak defuses,deaths per 10min,4-streak,headshots,6-streak,deaths,series id,bomb pickups,7-streak,kills per 10min,year,hill captures,kills,team kills,bomb defuses,2-piece,hits,match id,player,assists,fave weapon,8+-streak,end time,score
32,L,0,444.0,3sUp,"Scarab, Trinity Rocket, Bombardment",-4,0,Search & Destroy,0,14.4%,0,8,0.50,Throwback,2,0,0,0,111,0,10.8,0,1,0,8,champs-pool-A-2,0,0,5.4,2017,0,4,0,0,0,16,4061ACB6-111A-11D3-3BCD-00D9D1A0B6BB,Colechan,0,KBS Longbow,0,2017-08-09 14:25:42 UTC,2
33,L,0,444.0,3sUp,"Scarab, Trinity Rocket, Bombardment",-1,1,Search & Destroy,0,21.8%,0,8,0.86,Throwback,0,0,0,0,142,0,9.5,0,1,0,7,champs-pool-A-2,4,0,8.1,2017,0,6,0,0,1,31,4061ACB6-111A-11D3-3BCD-00D9D1A0B6BB,DraMa,2,KBAR-32,0,2017-08-09 14:25:42 UTC,2
34,L,0,444.0,3sUp,"Scarab, Trinity Rocket, Bombardment",-1,0,Search & Destroy,0,17.1%,0,8,0.83,Throwback,1,0,0,0,211,0,8.1,0,1,0,6,champs-pool-A-2,0,0,6.8,2017,0,5,0,0,0,36,4061ACB6-111A-11D3-3BCD-00D9D1A0B6BB,Mochila,1,NV4,0,2017-08-09 14:25:42 UTC,2
35,L,0,444.0,3sUp,"Scarab, Trinity Rocket, Bombardment",-3,0,Search & Destroy,0,28.4%,0,8,0.62,Throwback,1,0,0,0,102,0,10.8,0,0,0,8,champs-pool-A-2,0,0,6.8,2017,0,5,0,0,1,29,4061ACB6-111A-11D3-3BCD-00D9D1A0B6BB,Swarley,0,KBAR-32,0,2017-08-09 14:25:42 UTC,2
36,W,0,444.0,OpTic Gaming,"Scarab, Trinity Rocket, Bombardment",+2,0,Search & Destroy,0,27.9%,0,8,1.40,Throwback,1,0,0,0,111,0,6.8,1,0,0,5,champs-pool-A-2,0,0,9.5,2017,0,7,0,0,2,31,4061ACB6-111A-11D3-3BCD-00D9D1A0B6BB,Crimsix,3,Erad,0,2017-08-09 14:25:42 UTC,6
